## Pthreads API
* Q3: _What are all possible outputs produced by this program?_
    ```C
    1   #include <pthread.h>
    2   void *printer(void *arg) {
    3       char *ch = (char*)arg;
    4       printf("I am %c\n", *ch);
    5       return NULL;
    6   }
    7   int main() {
    8       pthread_t thread1, thread2;
    9       char *ch1 = malloc(sizeof(char));
    10      *ch1 = 'X';
    11      char *ch2 = malloc(sizeof(char));
    12      *ch2 = 'Y';
    13      pthread_create(&thread1, NULL, &printer, ch1);
    14      pthread_create(&thread2, NULL, &printer, ch2);
    15      pthread_join(thread1, NULL);
    16      pthread_join(thread2, NULL);
    17  }
    ```
    ```
    I am X	
    I am Y
    ```
    OR
    ```
	I am Y
	I am X
    ```

## Transforming programs to be multi-threaded

* For-each style loops are natural candidates for threading
    * Example: a program that counts the number of occurrences of each vowel within a string
    * Another example: a program that counts the longest sequence of each Short Tandem Repeat (STR) in a DNA sequence — you already performed loop optimization on such a program last week; this week in lab you'll make it multi-threaded
* Steps
    * Move the body of the loop into its own "worker" function (if not done already)
    * Write a "wrapper" function which:
        * Has the required signature for a thread function (i.e., takes a single `void *` parameter and returns a void pointer)
        * Extracts the argument(s) for the "worker" function from the thread function's arguments
        * Calls the "worker" function
        * Prepares the return value from the "worker" function to be returned by the thread
    * Replace the body of the loop with the creation of threads that execute the "wrapper" function
    * Add a second loop to wait for all threads to complete

## Making programs multi-threaded
* _Assume you are given the following code:_
    ```C
    #include <ctype.h>
    #include <stdio.h>
    #include <string.h>
    int count(char *str, char ch) {
        int num = 0;
        while (*str != '\0') {
            if (tolower(*str) == ch) {
                num++;
            }
            str++;
        }
        return num;
    }
    int main(int argc, char *argv[]) {
        if (argc < 2) {
            printf("Error: provide a string\n");
            return 1;
        }
        char *str = argv[1];
        char *vowels = "aeiou";
        int counts[strlen(vowels)];
        for (int i = 0; i < strlen(vowels); i++) {
            counts[i] = count(str, vowels[i]);
        }

        for (int i = 0; i < strlen(vowels); i++) {
            printf("%c %d\n", vowels[i], counts[i]);
        }
    }
    ```
* Q2: *Write a function called  `count_wrapper` that has the signature required for a thread function and calls the `count` function. (Hint: you'll need to declare a `struct` that contains all of the parameters required for `count`.)*
    ```C
    struct count_args {
        char *str;
        char ch;
    };
    void *count_wrapper(void *arg) {
        struct count_args *cargs = (struct count_args *)arg;
        int *result = malloc(sizeof(int));
        *result = count(cargs->str, cargs->ch);
        return result;
    }
    ```
* Q3: *Re-write `main` to create/wait for threads that execute `count_wrapper` (instead of calling `count` sequentially).*
    ```C
    int main(int argc, char *argv[]) {
        if (argc < 2) {
            printf("Error: provide a string\n");
            return 1;
        }
        char *str = argv[1];
        char *vowels = "aeiou";

        // Create threads
        struct count_args args[strlen(vowels)];
        pthread_t threads[strlen(vowels)];
        for (int i = 0; i < strlen(vowels); i++) {
            args[i].str = str;
            args[i].ch = vowels[i];
            pthread_create(&(threads[i]), NULL, &count_wrapper, &(args[i]));
        }

        // Wait for threads
        for (int i = 0; i < strlen(vowels); i++) {
            int *ptr;
            pthread_join(threads[i], (void **)(&ptr));
            printf("%c %d\n", vowels[i], *ptr);
        }
    }
    ```

* Q1: _The `compute_gcf` function below finds the greatest common factor between two numbers. Complete the `main` function to use threads to compute the greatest common factor for all pairs of numbers in the `to_factor` array. The program should produce the following output:_
    ```
    The GCF of 48 and 52 is 4
    The GCF of 25 and 100 is 25
    The GCF of 120 and 340 is 20
    ```

In [ ]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

void *compute_gcf(void *arg) {
    int *pair = (int *)arg;
    int *gcf = malloc(sizeof(int));
    *gcf = 1;
    for (int i = 2; i <= pair[0]; i++) {
        if (pair[0] % i == 0 && pair[1] % i == 0) {
            *gcf = i;
        }
    }
    return gcf;
}

#define NUM_PAIRS 3

int main() {
    int pairs[NUM_PAIRS][2] = { {48, 52}, {25, 100}, {120, 340} };

In [1]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

void *compute_gcf(void *arg) {
    int *pair = (int *)arg;
    int *gcf = malloc(sizeof(int));
    *gcf = 1;
    for (int i = 2; i <= pair[0]; i++) {
        if (pair[0] % i == 0 && pair[1] % i == 0) {
            *gcf = i;
        }
    }
    return gcf;
}

#define NUM_PAIRS 3

int main() {
    int pairs[NUM_PAIRS][2] = { {48, 52}, {25, 100}, {120, 340} };
    pthread_t threads[NUM_PAIRS];
    for (int i = 0; i < NUM_PAIRS; i++) {
        pthread_create(threads+i, NULL, &compute_gcf, pairs[i]);
    }
    for (int i = 0; i < NUM_PAIRS; i++) {
        int *gcf = NULL;
        pthread_join(threads[i], (void **)&gcf);
        printf("The GCF of %d and %d is %d\n", pairs[i][0], pairs[i][1], *gcf);
    }
}

The GCF of 48 and 52 is 4
The GCF of 25 and 100 is 25
The GCF of 120 and 340 is 20


## Extra practice

Q2: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *thread_main(void *arg) {
/* 4*/      char *id = (char *)arg;
/* 5*/      printf("I am thread %c\n", *id);
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  int main() {
/* 9*/      char *a = malloc(sizeof(char));
/*10*/      *a = 'A';
/*11*/      char *b = malloc(sizeof(char));
/*12*/      *b = 'B';
/*13*/      // Create thread running thread_main(a)
/*14*/      // Create thread running thread_main(b)
/*15*/      // Wait for threads to finish
/*16*/  }

```
I am thread A
I am thread B
```
OR
```
I am thread B
I am thread A
```

## Threads and memory

* Threads are multiple execution contexts within the **same process**
    * Processes are multiple execution contexts within the **same machine**
* Because threads are within the same process, they share all of the process’s resources — memory and CPU time
* Consequently, two threads can update the same variable

_Example_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *thread1_main(void *arg) {
/* 4*/      int *x = (int *)arg;
/* 5*/      *x += 1;
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  void *thread2_main(void *arg) {
/* 9*/      int *y = (int *)arg;
/*10*/      *y += 2;
/*11*/      return NULL;
/*12*/  }
/*13*/  int main() {
/*14*/      int *z = malloc(sizeof(int));
/*15*/      *z = 0;
/*16*/      // Create thread running thread1_main(z)
/*17*/      // Create thread running thread2_main(z)
/*18*/      // Wait for threads to finish
/*19*/      printf("z is %d\n", *z);
/*20*/  }

```
z is 3
```

<p style="height:31em;"></p>

Q1: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *increment(void *arg) {
/* 4*/      int *num = (int*)arg;
/* 5*/      *num += 1;
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  void *zero(void *arg) {
/* 9*/      int *num = (int*)arg;
/*10*/      *num = 0;
/*11*/      return NULL;
/*12*/  }
/*13*/  int main() {
/*14*/      int *i = malloc(sizeof(int));
/*15*/      *i = 5;
/*16*/      // Create thread running increment(i)
/*17*/      // Create thread running zero(i)
/*18*/      // Wait for threads to finish
/*19*/      printf("i=%d\n", *i);
/*20*/  }

```
i=0
```
OR
```
i=1
```

<p style="height:8em;"></p>

Q3: _What are all possible outputs produced by this program?_

In [1]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'P';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      pthread_join(thread1, NULL);
/*15*/      *ch = 'Q';
/*16*/      pthread_create(&thread2, NULL, &printer, ch);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

I am P
I am Q


<p style="height:8em;"></p>

Q4: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'M';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      *ch = 'N';
/*15*/      pthread_create(&thread2, NULL, &printer, ch);
/*16*/      pthread_join(thread1, NULL);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

```
I am M
I am N
```
OR
```
I am N
I am N
```

<div style="height:7em;"></div>

A program contains the following functions:
```C
void *dbl(void *arg) {
    int *t = (int *)arg;
    *t = *t * 2;
}

void *inc(void *arg) {
    int *t = (int *)arg;
    *t = *t + 1;
}
```
_For each of the following main methods, list **all possible outputs** the program could produce. Assume threads are only preempted if they become blocked waiting for other threads._

In [ ]:
// Q7
int main() {
    int *total = malloc(sizeof(int));
    *total = 3;
    pthread_t thrA, thrB;
    pthread_create(&thrA, NULL, &dbl, total);
    pthread_create(&thrB, NULL, &inc, total);
    pthread_join(thrA, NULL);
    pthread_join(thrB, NULL);
    printf("%d\n", total);
}

* `7` (if `thrB` runs after `thrA` finishes)
* `8` (if `thrA` runs after `thrB` finishes)

<p style="height:10em;"></p>

In [ ]:
// Q8
int main() {
    int *total = malloc(sizeof(int));
    *total = 3
    pthread_t thrA, thrB;
    pthread_create(&thrA, NULL, &inc, total);
    pthread_join(thrA, NULL);
    pthread_create(&thrB, NULL, &dbl, total);
    pthread_join(thrB, NULL);
    printf("%d\n", total);
}

* `8` (`thrA` is joined, i.e., must finish, before `thrB` is created) 

<p style="height:10em;"></p>

<p style="height:8em;"></p>